<a href="https://colab.research.google.com/github/katiko89/Educational-projects/blob/main/Python2%D0%9B%D0%A04%2C_%D0%9C%D0%B0%D1%80%D0%BA%D0%B8%D0%BD%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Загрузка данных
Загрузим данные из модуля torchvision

In [ ]:
import torch
from torchvision import datasets, transforms, models
import torch.nn as nn
import tqdm
from tqdm.auto import tqdm
import torch.optim as optim
import numpy as np

In [ ]:
transforms = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])

In [ ]:
train_data = datasets.MNIST(root="./mnist_data", train=True, download=True, transform=transforms)
test_data = datasets.MNIST(root="./mnist_data", train=False, download=True, transform=transforms)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=True)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:01<00:00, 5464210.23it/s]


Extracting ./mnist_data/MNIST/raw/train-images-idx3-ubyte.gz to ./mnist_data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 485638.38it/s]


Extracting ./mnist_data/MNIST/raw/train-labels-idx1-ubyte.gz to ./mnist_data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:01<00:00, 932927.50it/s] 


Extracting ./mnist_data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./mnist_data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 4477210.05it/s]

Extracting ./mnist_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./mnist_data/MNIST/raw



# I. Построение полносвязной нейронной сети
В этом раздеое реализуем свою архитектуру полносвязной нейронной сети с самостоятельно выбранным количеством слоев, нейронов в них и функций активации.

In [ ]:
class FullyConnected(nn.Module):
    def __init__(self):
        super(FullyConnected, self).__init__()
        self.fc1 = nn.Linear(28*28, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):
        x = x.view(-1, 28*28)
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = self.fc3(x)
        return x



#II. Построение свёрточной нейронной сети

In [ ]:
class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        #28x28
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=(3,3)) #26x26
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=16, kernel_size=(3,3)) #24x24
        self.pool2=nn.MaxPool2d(kernel_size=(2,2))#12x12
        self.conv3=nn.Conv2d(in_channels=16, out_channels=8, kernel_size=(3, 3)) #10x10
        # flatten
        self.flatten = nn.Flatten()

        self.fc1 = nn.Linear(10*10*8, 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, 10)


    def forward(self, x):
        x = nn.functional.relu(self.conv1(x))
        x = nn.functional.relu(self.conv2(x))
        x = nn.functional.relu(self.pool2(x))
        x = nn.functional.relu(self.conv3(x))

        x = self.flatten(x)

        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = self.fc3(x)

        return x

#III. Обучение сетей и применение существующих моделей для классификации изображений

In [ ]:
fc_model = FullyConnected()
cnn_model = ConvNet()

criterion = nn.CrossEntropyLoss()
optimizer_fc = optim.Adam(fc_model.parameters(), lr=0.001)
optimizer_cnn = optim.Adam(cnn_model.parameters(), lr=0.001)

In [ ]:
def train(model, optimizer, criterion, train_loader, epochs=10):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for i, data in enumerate(train_loader):
            inputs, labels = data
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f'For epoch {epoch+1} loss={running_loss/len(train_loader)}')

In [ ]:
train(fc_model, optimizer_fc, criterion, train_loader)

For epoch 1 loss=0.39627754677936977
For epoch 2 loss=0.19243448521933956
For epoch 3 loss=0.13774710511173138
For epoch 4 loss=0.11291494192992414
For epoch 5 loss=0.0943255129032957
For epoch 6 loss=0.08368378780982785
For epoch 7 loss=0.07413431728032352
For epoch 8 loss=0.06481606472553665
For epoch 9 loss=0.05994339001864921
For epoch 10 loss=0.05450525402073075


In [ ]:
train(cnn_model, optimizer_cnn, criterion, train_loader)

For epoch 1 loss=0.1902702399253992
For epoch 2 loss=0.05304972097027683
For epoch 3 loss=0.03530874703951212
For epoch 4 loss=0.027246409532675982
For epoch 5 loss=0.02060897485809933
For epoch 6 loss=0.017518146910886278
For epoch 7 loss=0.014222606642098849
For epoch 8 loss=0.011948412961102828
For epoch 9 loss=0.011608019800596614
For epoch 10 loss=0.010412381672140638


In [ ]:
from sklearn.metrics import accuracy_score

def evaluate(model, test_loader):

    true_labels = []
    predicted_labels = []

    with torch.no_grad():
        for images, labels in test_loader:
            X_test, y_test = images, labels
            outputs = model(X_test)
            _, pred = torch.max(outputs.data, 1)

            true_labels.extend(y_test.tolist())
            predicted_labels.extend(pred.tolist())

    accuracy = accuracy_score(true_labels, predicted_labels)
    print(f'Accuracy: {accuracy}')

evaluate(fc_model, test_loader)
evaluate(cnn_model, test_loader)

Accuracy: 0.975
Accuracy: 0.9896
